# Lesson 05 — Building Your First A2A Agent

Build a standalone agent with **multiple skills** that handles Q&A, claims filing
(multi-turn), and structured data output.
Supports **GitHub Models** (free, cloud) and **AI Toolkit LocalFoundry** (local,
no token required).

## What You'll Learn

- Select a model provider (GitHub Models or LocalFoundry)
- Configure an OpenAI-compatible API client
- Load domain knowledge and inject it into a system prompt
- Build a `QAAgent` async class for policy Q&A
- Build a `ClaimsAgent` for multi-turn claims filing
- Return **structured JSON** data (not just text)
- Route between multiple skills with `MultiSkillAgent`
- Test the agent standalone before adding A2A protocol layers

## Prerequisites

**GitHub Models** (default):

- GitHub account with a [Personal Access Token](https://github.com/settings/tokens) (no special scopes)
- `GITHUB_TOKEN` environment variable set in `.env`

**AI Toolkit LocalFoundry** (alternative — no token needed):

- VS Code with [AI Toolkit extension](https://marketplace.visualstudio.com/items?itemName=ms-windows-ai-studio.windows-ai-studio)
- A model loaded and running on port 5272

> **GitHub Models docs:** [docs.github.com/en/github-models](https://docs.github.com/en/github-models)


## Agent Architecture Overview

Before writing any code, here's what you're building and where it fits in the course.

```mermaid
graph TB
    subgraph Lesson05["Lesson 05 — This Notebook"]
        MS["MultiSkillAgent\n(router)"]
        QA["QAAgent\n(policy Q&A)"]
        CA["ClaimsAgent\n(multi-turn filing)"]
        PS["PolicySummaryAgent\n(structured JSON)"]
        MS -->|policy-qa| QA
        MS -->|claims-filing| CA
        MS -->|policy-summary| PS
    end

    subgraph Lesson06["Lesson 06 — A2A Server"]
        EX["InsuranceAgentExecutor\n(AgentExecutor)"]
        RH["DefaultRequestHandler"]
        APP["A2AStarletteApplication\n(port 10001)"]
        EX --> MS
        RH --> EX
        APP --> RH
    end

    subgraph Lesson07["Lesson 07 — A2A Client"]
        CL["A2AClient"]
        CR["A2ACardResolver"]
        CR --> APP
        CL --> APP
    end

    style Lesson05 fill:#e8f4f8,stroke:#4a90d9
    style Lesson06 fill:#fef9e7,stroke:#f39c12
    style Lesson07 fill:#e9f7ef,stroke:#27ae60
```

The three agent classes (`QAAgent`, `ClaimsAgent`, `PolicySummaryAgent`) are the **business logic**.
`MultiSkillAgent` is the router. In Lesson 06, `InsuranceAgentExecutor` wraps the router with
the A2A protocol interface, exposing it as a server that Lesson 07's client can discover and call.


## Step 1 — Install Dependencies


In [1]:
# ── Dependencies ──────────────────────────────────────────────────────────────
# openai        → OpenAI-compatible SDK (used to call GitHub Models / Phi-4)
# python-dotenv → Loads GITHUB_TOKEN from the .env file automatically
#
# If you have the venv active (a2a-examples kernel selected) these are already
# installed. Run this cell anyway to ensure the kernel is up to date.
# ─────────────────────────────────────────────────────────────────────────────
%pip install openai python-dotenv --quiet
print("Dependencies ready.")

Note: you may need to restart the kernel to use updated packages.
Dependencies ready.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import find_dotenv, load_dotenv

# ── Load secrets from .env ─────────────────────────────────────────────────
env_path = find_dotenv(raise_error_if_not_found=False)
if env_path:
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")
else:
    print("NOTE: .env not found — set GITHUB_TOKEN manually below if needed")

# ── Model provider ────────────────────────────────────────────────────────────
# "github"       — GitHub Models  (free, needs GITHUB_TOKEN in .env)
# "localfoundry" — AI Toolkit LocalFoundry  (local, no token needed)
PROVIDER = "github"  # ← change to "github" to use GitHub Models

if PROVIDER == "github":
    token = os.environ.get("GITHUB_TOKEN", "")
    if token:
        print(f"GITHUB_TOKEN is set ({token[:8]}...)")
    else:
        print("ERROR: GITHUB_TOKEN is NOT set — cells below will fail until you set it")
    ENDPOINT = "https://models.inference.ai.azure.com"
    API_KEY = token
    MODEL = "Phi-4"

elif PROVIDER == "localfoundry":
    ENDPOINT = os.environ.get("LOCALFOUNDRY_ENDPOINT", "http://localhost:5272/v1/")
    API_KEY = "unused"  # LocalFoundry ignores the key
    MODEL = os.environ.get("LOCALFOUNDRY_MODEL", "qwen2.5-0.5b-instruct-generic-gpu:4")
    print(f"NOTE: AI Toolkit LocalFoundry — ensure a model is running at {ENDPOINT}")

else:
    raise ValueError(f"Unknown PROVIDER: {PROVIDER!r}")

print(f"Provider  : {PROVIDER}")
print(f"Endpoint  : {ENDPOINT}")
print(f"Model     : {MODEL}")

Loaded .env from: y:\.sources\localm-tuts\a2a\_examples\.env
GITHUB_TOKEN is set (github_p...)
Provider  : github
Endpoint  : https://models.inference.ai.azure.com
Model     : Phi-4


## Step 2 — Configure the Model Client

Both **GitHub Models** and **AI Toolkit LocalFoundry** expose an **OpenAI-compatible API**.
We use the standard `openai` package — only `base_url` and `api_key` differ between providers.


In [3]:
from openai import AsyncOpenAI

client = AsyncOpenAI(
    base_url=ENDPOINT,
    api_key=API_KEY,
)

print(f"Client configured — base_url: {client.base_url}")
print(f"Model        : {MODEL}")

Client configured — base_url: https://models.inference.ai.azure.com
Model        : Phi-4


## Step 3 — Quick Model Test

Before building the agent, verify that the model connection works.


In [4]:
response = await client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "What is 2 + 2?"}],
    temperature=0.0,
)

print(f"Model: {response.model}")
print(f"Answer: {response.choices[0].message.content}")

Model: phi4
Answer: 2 + 2 equals 4.


## Step 4 — Load Domain Knowledge

The agent needs a knowledge base to answer domain-specific questions.
We load an insurance policy document and inject it into the system prompt.

This is **RAG-lite** — simple and effective for bounded domains.


In [5]:
from pathlib import Path

SYSTEM_PROMPT = """\
You are a helpful insurance policy assistant.
Use the following policy document to answer questions accurately.
If the answer is not in the document, say so clearly.
Always cite the relevant section when possible.

--- POLICY DOCUMENT ---
{policy_text}
--- END DOCUMENT ---
"""


def load_knowledge(path: str) -> str:
    """Load a knowledge document from disk."""
    return Path(path).read_text(encoding="utf-8")


# Load the insurance policy
knowledge = load_knowledge("data/insurance_policy.txt")
system_prompt = SYSTEM_PROMPT.format(policy_text=knowledge)

print(f"Loaded {len(knowledge)} characters of domain knowledge")
print(f"System prompt: {len(system_prompt)} characters")

Loaded 1763 characters of domain knowledge
System prompt: 2024 characters


## Step 5 — Build the QAAgent Class

The agent encapsulates:

- Model client (AsyncOpenAI)
- Model name (from `MODEL`)
- Domain knowledge (loaded from file)
- System prompt (template with injected knowledge)

**Key design decisions:**
| Decision | Choice | Rationale |
|---|---|---|
| Async interface | `async def query()` | A2A servers are async — start async from day one |
| Class pattern | `QAAgent` class | Clean separation for AgentExecutor wrapping (Lesson 6) |
| Low temperature | `0.2` | Factual Q&A needs deterministic responses |


In [6]:
class QAAgent:
    """Question-answering agent backed by the configured model provider."""

    def __init__(
        self,
        knowledge_path: str,
        model: str = MODEL,
        endpoint: str = ENDPOINT,
        api_key: str = API_KEY,
        temperature: float = 0.2,
    ):
        self.client = AsyncOpenAI(
            base_url=endpoint,
            api_key=api_key,
        )
        self.model = model
        self.temperature = temperature
        self.knowledge = load_knowledge(knowledge_path)
        self.system_prompt = SYSTEM_PROMPT.format(policy_text=self.knowledge)

    async def query(self, question: str) -> str:
        """Send a question to the model and return the answer."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": question},
            ],
            temperature=self.temperature,
            max_tokens=2048,
        )
        return response.choices[0].message.content


print("QAAgent class defined.")

QAAgent class defined.


## Step 6 — Test the QA Agent

**Always test standalone before wrapping in A2A.** This verifies:

- Model connectivity
- Knowledge injection
- Response quality


In [7]:
agent = QAAgent("data/insurance_policy.txt")
print(f"Agent created with {len(agent.knowledge)} chars of knowledge")

Agent created with 1763 chars of knowledge


In [8]:
# Test question 1: Specific fact
answer = await agent.query("What is the deductible for the Standard plan?")
print("Q: What is the deductible for the Standard plan?\n")
print(f"A: {answer}")

Q: What is the deductible for the Standard plan?

A: The deductible for the Standard plan includes:

- Standard Plan Deductible: $500 per incident
- Emergency Room Deductible: $250 per visit
- Prescription Drug Deductible: $100 per year

These details are found under the "DEDUCTIBLES" section of the policy document.


In [9]:
# Test question 2: Coverage question
answer = await agent.query("Are cosmetic procedures covered?")
print("Q: Are cosmetic procedures covered?\n")
print(f"A: {answer}")

Q: Are cosmetic procedures covered?

A: Cosmetic procedures are generally not covered under the ACME Insurance Standard Policy. The policy document states under the "EXCLUSIONS" section:

- Cosmetic procedures (unless medically necessary)

This means that cosmetic procedures are excluded from coverage unless they are deemed medically necessary. If you believe a procedure is medically necessary, you may need to provide documentation or justification to support your claim.


In [10]:
# Test question 3: Out-of-scope question (should say "not in document")
answer = await agent.query("What is the capital of France?")
print("Q: What is the capital of France?\n")
print(f"A: {answer}")

Q: What is the capital of France?

A: I'm sorry, but the information about the capital of France is not included in the policy document provided. However, I can tell you that the capital of France is Paris. If you have any questions related to the insurance policy document, feel free to ask!


---

## Step 7 — Build the ClaimsAgent (Multi-Turn)

The A2A protocol supports **multi-turn interactions** where an agent can
request additional input from the client mid-task (the `input_required` state).

The `ClaimsAgent` handles insurance claims filing:

1. **Check** if the user provided all required claim fields
2. **Ask** for missing fields (→ triggers INPUT_REQUIRED in Lesson 6)
3. **Process** the claim when all fields are present
4. **Return** structured JSON (claim receipt) — not just text

This demonstrates two key A2A concepts:

- **Multi-turn conversations** (§3.4 of the A2A spec)
- **Structured data exchange** (§6.8 of the A2A spec)


In [ ]:
import json
import re
from datetime import datetime, timezone
from uuid import uuid4

# Required fields for a valid insurance claim
REQUIRED_CLAIM_FIELDS = ["claim_type", "date_of_service", "amount", "description"]

CLAIMS_SYSTEM_PROMPT = """\
You are an insurance claims filing assistant. Extract claim data from user messages.

Required claim fields:
- claim_type: one of "medical", "dental", "prescription", "emergency"
- date_of_service: date in YYYY-MM-DD format
- amount: dollar amount as a string (e.g. "150.00")
- description: brief description of the service

STRICT RULES — you must follow these exactly:
- ONLY extract fields that are EXPLICITLY and CLEARLY stated by the user
- Do NOT infer, guess, assume, or fabricate any field value
- Do NOT add a field unless the user clearly provided that exact value
- If a field is absent or ambiguous, omit it from extracted_fields entirely
- Do NOT re-extract fields already collected (listed below)

Already collected fields (do NOT include these in extracted_fields):
{collected}

Return ONLY a JSON object with no markdown, no explanation:
{{
  "extracted_fields": {{"field_name": "value"}}
}}

If the user provided nothing extractable, return: {{"extracted_fields": {{}}}}
"""


class ClaimsAgent:
    """Multi-turn claims filing agent.

    Maintains state across turns to collect all required claim fields.
    Returns structured JSON with extracted fields and missing info.

    This pattern supports the A2A INPUT_REQUIRED task state:
    - Turn 1: User says "I need to file a claim for my dental visit"
    - Agent extracts claim_type=dental, asks for date, amount, description
    - Turn 2: User provides "$200 on 2025-01-15 for root canal"
    - Agent extracts all fields, processes the claim
    """

    def __init__(
        self,
        model: str = MODEL,
        endpoint: str = ENDPOINT,
        api_key: str = API_KEY,
    ):
        self.client = AsyncOpenAI(base_url=endpoint, api_key=api_key)
        self.model = model
        # Per-session state: task_id → collected fields
        self.sessions: dict[str, dict[str, str]] = {}

    async def process(self, user_text: str, session_id: str = "default") -> dict:
        """Process a claims turn. Returns a dict with status and data.

        Returns:
            {
                "status": "input_required" | "completed",
                "missing_fields": [...],        # if input_required
                "message": "...",               # human-readable
                "claim_receipt": {...}           # if completed
            }
        """
        collected = self.sessions.get(session_id, {})

        # Ask LLM to extract only explicitly stated fields from user message
        prompt = CLAIMS_SYSTEM_PROMPT.format(
            collected=json.dumps(collected) if collected else "(none yet)"
        )

        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_text},
            ],
            temperature=0.0,
            max_tokens=1024,
        )

        raw = response.choices[0].message.content

        # Parse the JSON from LLM response
        try:
            # Extract JSON from the response (LLM may wrap in markdown)
            json_match = re.search(r"\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}", raw, re.DOTALL)
            if json_match:
                parsed = json.loads(json_match.group())
            else:
                parsed = json.loads(raw)
        except json.JSONDecodeError:
            # Fallback: treat as no fields extracted
            parsed = {"extracted_fields": {}}

        # Merge extracted fields into session state
        extracted = parsed.get("extracted_fields", {})
        collected.update({k: v for k, v in extracted.items() if v})
        self.sessions[session_id] = collected

        # Determine what's still missing — computed deterministically in Python
        missing = [f for f in REQUIRED_CLAIM_FIELDS if f not in collected]

        if missing:
            return {
                "status": "input_required",
                "collected_fields": collected,
                "missing_fields": missing,
                "message": f"Please provide: {', '.join(missing)}",
            }
        else:
            # All fields collected — generate claim receipt
            receipt = self._generate_receipt(collected, session_id)
            # Clear session
            del self.sessions[session_id]
            return {
                "status": "completed",
                "claim_receipt": receipt,
                "message": f"Claim {receipt['claim_id']} filed successfully.",
            }

    def clear_session(self, session_id: str) -> None:
        """Remove a claims session (used by cancel)."""
        self.sessions.pop(session_id, None)

    def _generate_receipt(self, fields: dict, session_id: str) -> dict:
        """Generate a structured claim receipt (JSON artifact)."""
        return {
            "claim_id": f"CLM-{uuid4().hex[:8].upper()}",
            "policy_number": "ACME-STD-2025-001",
            "claim_type": fields.get("claim_type", "unknown"),
            "date_of_service": fields.get("date_of_service", "unknown"),
            "amount": fields.get("amount", "0.00"),
            "description": fields.get("description", ""),
            "status": "submitted",
            "filed_at": datetime.now(timezone.utc).isoformat(),
            "estimated_processing_days": 30,
        }


print("ClaimsAgent class defined.")

ClaimsAgent class defined.


## Step 8 — Test the ClaimsAgent (Multi-Turn)

Simulate a multi-turn conversation:

1. User provides partial info → agent asks for more
2. User provides remaining info → agent files the claim


In [12]:
claims_agent = ClaimsAgent()

# Turn 1: Partial info
result1 = await claims_agent.process(
    "I need to file a dental claim for a root canal", session_id="demo-1"
)
print("Turn 1 result:")
print(json.dumps(result1, indent=2))
print()

Turn 1 result:
{
  "status": "input_required",
  "collected_fields": {
    "claim_type": "dental"
  },
  "missing_fields": [
    "date_of_service",
    "amount",
    "description"
  ],
  "message": "Please provide: date_of_service, amount, description"
}



In [13]:
# Turn 2: Provide the remaining info (date, amount, and description — all at once)
result2 = await claims_agent.process(
    "The visit was on 2025-01-15 and it cost $450 for a root canal procedure",
    session_id="demo-1",
)
print("Turn 2 result:")
print(json.dumps(result2, indent=2))

Turn 2 result:
{
  "status": "completed",
  "claim_receipt": {
    "claim_id": "CLM-1F75BA9A",
    "policy_number": "ACME-STD-2025-001",
    "claim_type": "dental",
    "date_of_service": "2025-01-15",
    "amount": "450.00",
    "description": "root canal procedure",
    "status": "submitted",
    "filed_at": "2026-02-28T21:39:44.748664+00:00",
    "estimated_processing_days": 30
  },
  "message": "Claim CLM-1F75BA9A filed successfully."
}


Notice:

- Turn 1 returned `"status": "input_required"` with a list of `missing_fields`
- Turn 2 returned `"status": "completed"` with a full `claim_receipt` — structured JSON

The A2A protocol maps this directly:

- `input_required` → `TaskState.input_required` (server pauses, waits for client follow-up)
- `completed` → `TaskState.completed` with an `Artifact` containing the receipt JSON


### How Multi-Turn Maps to the A2A Protocol

```mermaid
sequenceDiagram
    participant User
    participant ClaimsAgent
    participant Session as "sessions dict\n(in-memory state)"

    User->>ClaimsAgent: "file a dental claim"
    ClaimsAgent->>Session: get collected = {}
    ClaimsAgent-->>User: status=input_required<br/>missing=[date, amount, description]

    Note over ClaimsAgent,Session: Session stores claim_type=dental

    User->>ClaimsAgent: "root canal on 2025-01-15 for $450"
    ClaimsAgent->>Session: get collected = {claim_type: dental}
    ClaimsAgent->>Session: merge: date=2025-01-15, amount=$450
    Note over ClaimsAgent: still missing: description

    ClaimsAgent-->>User: status=input_required<br/>missing=[description]

    User->>ClaimsAgent: "root canal procedure"
    ClaimsAgent->>Session: merge: description=root canal procedure
    Note over ClaimsAgent: ALL fields present → generate receipt
    ClaimsAgent->>Session: del session (cleanup)
    ClaimsAgent-->>User: status=completed<br/>claim_receipt={claim_id, ...}
```

In Lesson 06 these translate directly to A2A protocol events emitted by `InsuranceAgentExecutor`:

- `input_required` → `TaskStatusUpdateEvent(state=TaskState.input_required)`
- `completed` → `TaskArtifactUpdateEvent(artifact=Artifact(parts=[DataPart(data=receipt)]))`


---

## Step 9 — Build the PolicySummaryAgent (Structured Data)

The A2A protocol supports **structured data parts** (`DataPart`) for machine-readable
exchange. This agent returns policy summaries as **JSON artifacts** rather than prose.

This demonstrates A2A **Artifact** output (§4.1.7) and **DataPart** (§4.1.6).


In [14]:
class PolicySummaryAgent:
    """Returns structured policy summaries as JSON.

    Instead of free-text answers, this agent returns machine-readable
    data that can be consumed by other agents or UIs.
    """

    def __init__(
        self,
        knowledge_path: str = "data/insurance_policy.txt",
        model: str = MODEL,
        endpoint: str = ENDPOINT,
        api_key: str = API_KEY,
    ):
        self.client = AsyncOpenAI(base_url=endpoint, api_key=api_key)
        self.model = model
        self.knowledge = load_knowledge(knowledge_path)

    async def summarize(self) -> dict:
        """Generate a structured policy summary."""
        prompt = f"""\
Extract the key facts from this insurance policy and return ONLY a JSON object:

{self.knowledge}

Return this exact JSON structure (fill in values from the document):
{{
  "policy_number": "...",
  "plan_name": "...",
  "monthly_premium": "...",
  "annual_premium": "...",
  "deductible": "...",
  "annual_maximum": "...",
  "covered_services": ["..."],
  "exclusions": ["..."]
}}
"""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=2048,
        )

        raw = response.choices[0].message.content

        try:
            json_match = re.search(r"\{.*\}", raw, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
            return json.loads(raw)
        except json.JSONDecodeError:
            return {"error": "Failed to parse structured output", "raw": raw}


print("PolicySummaryAgent class defined.")

PolicySummaryAgent class defined.


In [15]:
# Test the structured output agent
summary_agent = PolicySummaryAgent()
summary = await summary_agent.summarize()

print("Policy Summary (structured JSON):")
print(json.dumps(summary, indent=2))

Policy Summary (structured JSON):
{
  "policy_number": "ACME-STD-2025-001",
  "plan_name": "Standard",
  "monthly_premium": "$150",
  "annual_premium": "$1,800",
  "deductible": {
    "standard_plan": "$500 per incident",
    "emergency_room": "$250 per visit",
    "prescription_drug": "$100 per year"
  },
  "annual_maximum": "$200,000",
  "covered_services": [
    "Primary Care Visits: Covered after deductible, 80/20 co-insurance",
    "Specialist Visits: Covered after deductible, 70/30 co-insurance",
    "Emergency Room: Covered after ER deductible, 90/10 co-insurance",
    "Prescription Drugs: Generic: $10 co-pay, Brand Name: $30 co-pay, Specialty: 20% co-insurance after drug deductible",
    "Preventive Care: Covered at 100%, no deductible",
    "Mental Health: Covered after deductible, 80/20 co-insurance",
    "Physical Therapy: Up to 30 visits per year, $25 co-pay per visit"
  ],
  "exclusions": [
    "Cosmetic procedures (unless medically necessary)",
    "Experimental treatment

---

## Step 10 — Multi-Skill Routing

Real A2A agents declare **multiple skills** in their Agent Card. The server
routes requests to the appropriate skill based on the message content.

The `MultiSkillAgent` wraps all three agents and routes based on intent:

| Skill            | Agent              | Trigger Pattern                                             |
| ---------------- | ------------------ | ----------------------------------------------------------- |
| `policy-qa`      | QAAgent            | questions about coverage, deductibles, premiums             |
| `claims-filing`  | ClaimsAgent        | "file a dental claim", "submit claim", "medical claim" etc. |
| `policy-summary` | PolicySummaryAgent | "summary", "summarize", "overview"                          |


### Skill Routing Flow

```mermaid
flowchart TD
    Input["User message"] --> Router{"MultiSkillAgent\n.detect_skill()"}

    Router -->|regex: 'file.*claim'\n'dental claim'\n'submit claim'| Claims["ClaimsAgent\n.process()"]
    Router -->|contains 'summary'\n'summarize'\n'overview'| Summary["PolicySummaryAgent\n.summarize()"]
    Router -->|everything else| QA["QAAgent\n.query()"]

    Claims --> CheckComplete{"All 4 fields\ncollected?"}
    CheckComplete -->|No| InputRequired["status: input_required\nmissing_fields: [...]"]
    CheckComplete -->|Yes| ClaimReceipt["status: completed\nclaim_receipt: {...}"]

    Summary --> StructuredJSON["status: completed\ndata: {policy JSON}"]
    QA --> TextAnswer["status: completed\nanswer: '...'"]
```

**Why skill detection matters for A2A:**
The Lesson 06 `InsuranceAgentExecutor` calls `detect_skill()` on every message, then stores the
active skill in `self.active_skills[task_id]` so follow-up messages in a multi-turn conversation
continue routing to the same skill.


In [ ]:
class MultiSkillAgent:
    """Routes requests to the appropriate skill agent.

    In Lesson 6, the A2A Agent Card declares these skills, and the
    AgentExecutor uses this router to handle incoming messages.
    """

    CLAIM_PATTERN = re.compile(
        r"\b(file|submit|make|start)\b.*\bclaims?\b"
        r"|\bclaims?\s+(for|about|regarding)\b"
        r"|\b(dental|medical|prescription|emergency)\s+claims?\b",
        re.IGNORECASE,
    )
    SUMMARY_KEYWORDS = ["summary", "summarize", "overview", "key facts"]

    def __init__(self, knowledge_path: str = "data/insurance_policy.txt"):
        self.qa_agent = QAAgent(knowledge_path)
        self.claims_agent = ClaimsAgent()
        self.summary_agent = PolicySummaryAgent(knowledge_path)

    def detect_skill(self, user_text: str) -> str:
        """Detect which skill to route to based on user input."""
        lower = user_text.lower()
        if self.CLAIM_PATTERN.search(lower):
            return "claims-filing"
        if any(kw in lower for kw in self.SUMMARY_KEYWORDS):
            return "policy-summary"
        return "policy-qa"

    async def handle(
        self, user_text: str, skill: str | None = None, session_id: str = "default"
    ) -> dict:
        """Handle a request by routing to the correct skill.

        Returns:
            {"skill": str, "type": "text"|"structured"|"multi_turn", ...}
        """
        detected_skill = skill or self.detect_skill(user_text)

        if detected_skill == "claims-filing":
            result = await self.claims_agent.process(user_text, session_id)
            return {"skill": "claims-filing", "type": "multi_turn", **result}

        elif detected_skill == "policy-summary":
            summary = await self.summary_agent.summarize()
            return {
                "skill": "policy-summary",
                "type": "structured",
                "status": "completed",
                "data": summary,
            }

        else:  # policy-qa
            answer = await self.qa_agent.query(user_text)
            return {
                "skill": "policy-qa",
                "type": "text",
                "status": "completed",
                "answer": answer,
            }


print("MultiSkillAgent class defined.")

MultiSkillAgent class defined.


In [17]:
multi = MultiSkillAgent()

# Test skill routing
tests = [
    "What is the monthly premium?",  # → policy-qa
    "I need to file a claim for a dental visit",  # → claims-filing
    "Give me a summary of my policy",  # → policy-summary
]

for text in tests:
    skill = multi.detect_skill(text)
    print(f"  '{text[:50]}...' → {skill}")

  'What is the monthly premium?...' → policy-qa
  'I need to file a claim for a dental visit...' → claims-filing
  'Give me a summary of my policy...' → policy-summary


In [18]:
# Exercise all three skills

# 1. Policy Q&A
r1 = await multi.handle("What is the deductible?")
print(f"Skill: {r1['skill']} | Type: {r1['type']}")
print(f"Answer: {r1['answer'][:100]}...")
print()

# 2. Claims Filing — multi-turn
r2 = await multi.handle(
    "I need to file a claim for dental work on 2025-02-01 — $300 for a crown",
    session_id="test-claim",
)
print(f"Skill: {r2['skill']} | Type: {r2['type']} | Status: {r2['status']}")
if r2["status"] == "completed":
    print(f"Claim Receipt: {json.dumps(r2['claim_receipt'], indent=2)}")
else:
    print(f"Missing: {r2['missing_fields']}")
print()

# 3. Policy Summary — structured data
r3 = await multi.handle("Give me a policy summary")
print(f"Skill: {r3['skill']} | Type: {r3['type']}")
print(f"Data: {json.dumps(r3['data'], indent=2)[:200]}...")

Skill: policy-qa | Type: text
Answer: The policy outlines several deductibles:

1. **Standard Plan Deductible**: $500 per incident.
2. **E...

Skill: claims-filing | Type: multi_turn | Status: completed
Claim Receipt: {
  "claim_id": "CLM-41BD63ED",
  "policy_number": "ACME-STD-2025-001",
  "claim_type": "dental",
  "date_of_service": "2025-02-01",
  "amount": "300",
  "description": "a crown",
  "status": "submitted",
  "filed_at": "2026-02-28T21:40:40.623841+00:00",
  "estimated_processing_days": 30
}

Skill: policy-summary | Type: structured
Data: {
  "policy_number": "ACME-STD-2025-001",
  "plan_name": "Standard",
  "monthly_premium": "$150",
  "annual_premium": "$1,800",
  "deductible": {
    "standard_plan": "$500 per incident",
    "emergen...


---

## Step 11 — Experiment

Try these variations:

- Change the `temperature` (0.0 for max determinism, 0.8 for more creativity)
- Try a multi-turn claims conversation with partial info each turn
- Ask the summary agent and compare with the Q&A agent
- Modify `CLAIM_PATTERN` regex to test skill routing


In [ ]:
# Try your own question!
your_question = "How much is the monthly premium?"

result = await multi.handle(your_question)
print(f"Skill: {result['skill']}")
print(f"Result: {json.dumps(result, indent=2, default=str)[:500]}")

Skill: policy-qa
Result: {
  "skill": "policy-qa",
  "type": "text",
  "status": "completed",
  "answer": "The monthly premium for the Standard Plan is $150. This information is found in the \"COVERAGE SUMMARY\" section of the policy document."
}


: 

## A2A Protocol Coverage — What This Agent Enables

| A2A Feature              | Agent Capability          | Lesson 6 Maps To           |
| ------------------------ | ------------------------- | -------------------------- |
| Multi-Turn (§3.4)        | ClaimsAgent sessions      | `TaskState.input_required` |
| Artifacts (§4.1.7)       | Structured claim receipts | `TaskArtifactUpdateEvent`  |
| DataPart (§4.1.6)        | JSON policy summaries     | `DataPart` in artifacts    |
| Multiple Skills (§4.4.5) | 3-skill router            | Agent Card `skills[]`      |
| Task Lifecycle (§4.1.3)  | Status tracking           | `working` → `completed`    |
| Text Parts (§4.1.6)      | Q&A answers               | `TextPart` in messages     |

## Next Steps

This multi-skill agent is the **foundation** for the A2A server.

- **Lesson 6** → Wrap with AgentExecutor + Rich Agent Card (3 skills, streaming, artifacts)
- **Lesson 7** → Client exercises all capabilities: multi-turn, artifacts, task management
